# Tutorial : TermSheetTransformer

Ce notebook présente comment utiliser **TermSheetTransformer** pour modifier des documents Word de type term sheets et générer automatiquement trois versions :

1. **Mark-up** : document avec les modifications visibles (jaune + barré)
2. **Final** : document modifié sans marquage
3. **PDF** : version PDF du document final

## 1. Installation et import

In [ ]:
# Installation des dépendances (à exécuter une fois)
# !pip install python-docx docx2pdf

from ts_editor_D import TermSheetTransformer
from pathlib import Path

## 2. Initialisation

On crée un transformer en passant le chemin vers le document Word source.

In [ ]:
# Chemin vers votre document Word (.docx)
CHEMIN_DOCUMENT = "mon_term_sheet.docx"

transformer = TermSheetTransformer(CHEMIN_DOCUMENT)

## 3. Déclaration des modifications

L'API est **fluide** : chaque méthode retourne `self`, ce qui permet d'enchaîner les appels.

### 3.1 Remplacement de mots

Remplace toutes les occurrences d'un mot par un autre dans tout le document.

In [ ]:
transformer.replace_word("Mot1", "Mot2")
transformer.replace_word("Ancienne valeur", "Nouvelle valeur")

### 3.2 Ajout d'une section

Insère une nouvelle ligne (section) après une section existante, en conservant le style.

In [ ]:
transformer.add_section_after(
    after_section="Listing",      # Titre de la section après laquelle insérer
    title="Issuer",                # Titre de la nouvelle section
    description="Paul Berber"      # Description
)

### 3.3 Modification d'une description

Modifie la description d'une section existante (cellule de droite).

In [ ]:
transformer.update_section_description(
    section_title="Country",
    new_description="France"
)

### 3.4 Suppression d'une section

In [ ]:
transformer.remove_section("Old Section")

### 3.5 Ajout du logo

Ajoute un logo en haut à gauche de chaque page du document.

In [ ]:
transformer.add_logo(
    logo_path="logo_nbc.png",
    width_inches=0.8
)

## 4. Exécution et export

Une fois toutes les modifications déclarées, on lance l'exécution. Les trois fichiers sont générés dans le dossier de sortie.

In [ ]:
transformer.execute_and_export(
    output_dir="./output",
    markup_docx="modifications.docx",   # Version avec modifications visibles
    final_docx="document_final.docx",  # Version propre
    final_pdf="document_final.pdf"     # PDF (nécessite Word installé)
)

## 5. Règles de marquage (version Mark-up)

Dans le document **modifications.docx**, les changements sont affichés ainsi :

| Type de modification | Rendu |
|---------------------|-------|
| **Remplacement** (Mot1 → Mot2) | Mot1 barré + Mot2 surligné jaune |
| **Ajout** de section | Contenu surligné jaune |
| **Modification** de description | Nouvelle description surlignée jaune |
| **Suppression** | Texte barré (conservé pour trace) |

## 6. Exemple complet en une seule fois

In [ ]:
# Réinitialiser et tout faire en chaîne
transformer = (
    TermSheetTransformer(CHEMIN_DOCUMENT)
    .replace_word("Mot1", "Mot2")
    .add_section_after("Listing", "Issuer", "Paul Berber")
    .update_section_description("Country", "France")
    .remove_section("Section obsolète")
    .add_logo("logo_nbc.png", width_inches=0.8)
)

transformer.execute_and_export(
    output_dir="./output",
    markup_docx="modifications.docx",
    final_docx="document_final.docx",
    final_pdf="document_final.pdf"
)

## 7. Paramètre `occurrence`

Si une section apparaît plusieurs fois dans le document (même titre), on peut cibler la n-ième occurrence :

In [ ]:
# Cibler la 2ème section "Listing"
transformer.add_section_after("Listing", "Nouvelle section", "Description", occurrence=2)

# Modifier la 1ère section "Country" (par défaut)
transformer.update_section_description("Country", "France", occurrence=1)

## 8. Gestion des erreurs

Si une section est introuvable, une `ValueError` est levée :

In [ ]:
try:
    transformer = TermSheetTransformer(CHEMIN_DOCUMENT)
    transformer.update_section_description("Section inexistante", "Nouvelle valeur")
    transformer.execute_and_export("./output")
except ValueError as e:
    print(f"Erreur : {e}")

---

**Note** : La conversion PDF nécessite Microsoft Word installé sur la machine (via `docx2pdf`). Si la conversion échoue, les fichiers Word sont tout de même générés.